In [1]:
import sys, time, pickle
import torch

sys.path.insert(0, '../../Utils')
from global_constants import gpu_device

sys.path.insert(0, '../')
from BERT_models import BERT_QNLI_MODEL

from _utils import sample_random_glue_qnli, get_continuation_mapping, \
                    get_continuous_attributions, get_continuous_raw_inputs, \
                    collect_info_for_metric, save_info, download_HTML

In [2]:
qnli_data_raw, targets, idxs = sample_random_glue_qnli()

Reusing dataset glue (/home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-e6d0a9c9270b6822.arrow
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-c46c5e78d2bda808.arrow


In [3]:
model = BERT_QNLI_MODEL().to('cuda:1')
tokenizer = model.tokenizer

In [4]:
#define some containers to save some info
model_out_list, raw_attr_list, conti_attr_list, raw_input_list = [], [], [], []

In [5]:
from captum.attr import KernelShap
from captum.attr import visualization 

In [6]:
ks = KernelShap(model)

In [7]:
def generate_record(raw_datum, target): #raw_datum expected to be a tuple/list of 2 sentences
    #tokenizer operations
    tokenized = tokenizer(raw_datum, truncation=True, return_offsets_mapping=True)
    offset_mappings = tokenized['offset_mapping']
    #concatenate the two offset_mappings together because they are fed in together
    conti_map = get_continuation_mapping(offset_mappings[0]) + get_continuation_mapping(offset_mappings[1])
    #change the first input_id of the second sentence to be the last input_id of the 1st sentence (i.e. an [END] token))
    tokenized_input_ids = tokenized['input_ids'][0] + \
                        [tokenized['input_ids'][1][i] if i != 0 else tokenized['input_ids'][0][-1] \
                         for i in range(len(tokenized['input_ids'][1]))]
    input_ids = torch.tensor(tokenized_input_ids).unsqueeze(0).to('cuda:1') #to(gpu_device)
    detokenized = [t.replace('#', '') for t in tokenizer.convert_ids_to_tokens(input_ids[0])]
    
    #feeding input forward 
    input_emb = model.get_embeddings(input_ids)
    pred_prob = model(input_emb).item()

     #categorizing results
    pred_class = 'No Entailment' if pred_prob < 0.5 else 'Entailment' 
    true_class = 'No Entailment' if target < 0.5 else 'Entailment' 
    
    #attribution algorithm working
    attribution = ks.attribute(input_emb, n_samples=5000, perturbations_per_eval=200, show_progress=True)
    word_attributions = attribution.squeeze(0).sum(dim=1)
#     word_attributions = attr_normalizing_func(word_attributions)
    word_attributions /= torch.norm(word_attributions)
    attr_score = torch.sum(word_attributions)
    attr_class = 'No Entailment' if attr_score < 0.5 else 'Entailment'
    convergence_score = None
    
    
#     #re-organizing tensors and arrays because words get split down
    conti_attr = get_continuous_attributions(conti_map, word_attributions)
    raw_input = get_continuous_raw_inputs(conti_map, detokenized)

#     print(f'word attributions {word_attributions}')
    print(f'pred_prob {pred_prob}')
#     print(f'pred_class {pred_class}')
#     print(f'true_class {true_class}')
#     print(f'attribution {attribution}')
#     print(f'attr_class {attr_class}')
#     print(f'attr_score {attr_score}')
#     print(f'raw_input {raw_input}')

        
# #     collect info for metrics later
    collect_info_for_metric(model_out_list, pred_prob, raw_attr_list, attribution, conti_attr_list, conti_attr, raw_input_list, raw_input)
        
    
    visual_record = visualization.VisualizationDataRecord(word_attributions=conti_attr,
                                                         pred_prob=pred_prob,
                                                         pred_class=pred_class,
                                                         true_class=true_class,
                                                         attr_class=attr_class,
                                                         attr_score=attr_score,
                                                         raw_input_ids=raw_input,
                                                         convergence_score=convergence_score)
        
        
    return visual_record
      
    

In [8]:
for i, (datum_raw, target) in enumerate(zip(qnli_data_raw, targets), start=1):
#     example_1 = "How many times has the South Florida/Miami area hosted the Super Bowl?"
#     example_2 = "The South Florida/Miami area has previously hosted the event 10 times (tied for most with New Orleans), with the most recent one being Super Bowl XLIV in 2010."
    
#     example_1 = "When did the third Digimon series begin?"
#     example_2 = "Unlike the two seasons before it and most of the seasons that followed, Digimon Tamers takes a darker and more realistic approach to its story featuring Digimon who do not reincarnate after their deaths and more complex character development in the original Japanese."
#     datum_raw, target = [example_1, example_2], 1
    print(f'Raw datum: {datum_raw}') #datum expected to be a list of 2 sentences
    print(f'GT target: {target}')
    visual_record=generate_record(datum_raw, target)
    print(visualization.visualize_text([visual_record])) 


Raw datum: ['What would a teacher assess the levels of a student on?', 'For example, an experienced teacher and parent described the place of a teacher in learning as follows: "The real bulk of learning takes place in self-study and problem solving with a lot of feedback around that loop.']
GT target: 0


/home/user/anaconda3/envs/metric/lib/python3.9/site-packages/captum/attr/_core/lime.py:1096: UserWarning: Attempting to construct interpretable model with > 10000 features.This can be very slow or lead to OOM issues. Please provide a featuremask which groups input features to reduce the number of interpretablefeatures. 
  warnings.warn(
Kernel Shap attribution: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:11<00:00,  2.23it/s]


word attr tensor([ 0.0739, -0.0500, -0.0031,  0.2475,  0.1116, -0.0299, -0.0603,  0.0788,
        -0.1102,  0.1157,  0.1900,  0.0360,  0.1701,  0.2313,  0.2107,  0.1070,
         0.0557,  0.0072,  0.0033, -0.1602, -0.0663, -0.0480, -0.1066, -0.1058,
         0.1902,  0.1391,  0.2323,  0.0098,  0.0864, -0.0510,  0.0095, -0.0715,
         0.0992, -0.1142,  0.1759, -0.1886,  0.2088,  0.0996, -0.0835,  0.1667,
         0.1622, -0.1074,  0.0841,  0.1214, -0.0655,  0.0863, -0.0237,  0.1670,
         0.0615, -0.1404,  0.1536,  0.2482,  0.0392,  0.2646, -0.0884, -0.0821,
         0.0011,  0.0427, -0.2220], device='cuda:1')
conti attr [tensor(0.0739, device='cuda:1'), tensor(-0.0500, device='cuda:1'), tensor(-0.0031, device='cuda:1'), tensor(0.2475, device='cuda:1'), tensor(0.1116, device='cuda:1'), tensor(-0.0299, device='cuda:1'), tensor(-0.0603, device='cuda:1'), tensor(0.0788, device='cuda:1'), tensor(-0.1102, device='cuda:1'), tensor(0.1157, device='cuda:1'), tensor(0.1900, device='cuda:1'

/home/user/anaconda3/envs/metric/lib/python3.9/site-packages/captum/attr/_core/lime.py:1096: UserWarning: Attempting to construct interpretable model with > 10000 features.This can be very slow or lead to OOM issues. Please provide a featuremask which groups input features to reduce the number of interpretablefeatures. 
  warnings.warn(


<IPython.core.display.HTML object>
Raw datum: ['What company created Doctor Who?', 'Who character by BBC Television in the early 1960s, a myriad of stories have been published about Doctor Who, in different media: apart from the actual television episodes that continue to be produced by the BBC, there have also been novels, comics, short stories, audio books, radio plays, interactive video games, game books, webcasts, DVD extras, and even stage performances.']
GT target: 1


...

/home/user/anaconda3/envs/metric/lib/python3.9/site-packages/captum/attr/_core/lime.py:1096: UserWarning: Attempting to construct interpretable model with > 10000 features.This can be very slow or lead to OOM issues. Please provide a featuremask which groups input features to reduce the number of interpretablefeatures. 
  warnings.warn(


<IPython.core.display.HTML object>
Raw datum: ['If the apparant force of two fermions is attractive, what is the spin function?', 'If two identical fermions (e.g. electrons) have a symmetric spin function (e.g. parallel spins) the spatial variables must be antisymmetric (i.e. they exclude each other from their places much as if there was a repulsive force), and vice versa, i.e. for antiparallel spins the position variables must be symmetric (i.e. the apparent force must be attractive).']
GT target: 1


Kernel Shap attribution: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:17<00:00,  1.43it/s]


word attr tensor([ 0.0657, -0.0076,  0.0856,  0.1123, -0.1251,  0.1080, -0.1925,  0.1691,
        -0.0444,  0.1078,  0.0173,  0.0137, -0.0225,  0.0019, -0.0386, -0.0397,
        -0.0952, -0.0450, -0.0502,  0.0808,  0.0221,  0.0696,  0.0864, -0.1319,
         0.0319,  0.0686, -0.0405, -0.0961, -0.0609, -0.1122, -0.0520,  0.0470,
        -0.0758, -0.0455, -0.0268,  0.0536, -0.0098,  0.1566,  0.0192,  0.2612,
         0.0231, -0.0020, -0.1150,  0.0015, -0.0697,  0.1908, -0.1155,  0.0562,
         0.0309, -0.1356,  0.0095, -0.1789, -0.0103, -0.1876,  0.0958,  0.0088,
         0.0064, -0.1232, -0.1233,  0.2519, -0.0628, -0.0011,  0.0732, -0.0232,
        -0.0700,  0.1096,  0.0381,  0.1213,  0.0325, -0.0897,  0.0582,  0.1320,
        -0.0316, -0.0417, -0.0496, -0.0745,  0.0395,  0.0467,  0.0307, -0.0168,
        -0.1117,  0.0672,  0.1109,  0.0932, -0.0599, -0.1002,  0.0247,  0.1735,
         0.0073,  0.0085, -0.0726, -0.0336,  0.0431, -0.0244, -0.1330, -0.0072,
        -0.1016,  0.1202, -0.0

<IPython.core.display.HTML object>


In [9]:
file_name_base = 'Kernel_SHAP'

In [10]:
save_info(idxs, qnli_data_raw, targets, model_out_list, raw_attr_list, conti_attr_list, raw_input_list, 
          fname=f'../MethodOutputs/{file_name_base}_out.pkl')

{'indices': [4013,
  2281,
  2771,
  5109,
  1852,
  2016,
  642,
  3912,
  4427,
  2330,
  4581,
  2611,
  5186,
  2700,
  4920,
  4335,
  1246,
  2811,
  5290,
  5385,
  4774,
  653,
  2816,
  916,
  3297,
  3459,
  1475,
  1287,
  4548,
  1476,
  2328,
  2208,
  2772,
  5301,
  869,
  4968,
  4456,
  4711,
  1929,
  634,
  3774,
  5389,
  497,
  4901,
  4602,
  831,
  1292,
  1087,
  3159,
  3492],
 'raw_data': [['What would a teacher assess the levels of a student on?',
   'For example, an experienced teacher and parent described the place of a teacher in learning as follows: "The real bulk of learning takes place in self-study and problem solving with a lot of feedback around that loop.'],
  ['What company created Doctor Who?',
   'Who character by BBC Television in the early 1960s, a myriad of stories have been published about Doctor Who, in different media: apart from the actual television episodes that continue to be produced by the BBC, there have also been novels, comics, sho

In [11]:
download_HTML(cur_file_name=f'{file_name_base}_QNLI_BERT.ipynb',
              out_file_name=f'{file_name_base}_QNLI_BERT.html')